# Janus

In [215]:
import PIL # reminder that this is a requirement
from scipy.misc import imread
import matplotlib.pyplot as plt
#from matplotlib.patches import Ellipse

def write_png(params):
    sim = rebound.Simulation.from_file(params['filename'])
    ps = sim.particles
    plotparticles = params['plotparticles']       # particles that should be displayed
    plottransits = params['plottransits']         # whether we should highlight transits
    transittargets =params["transittargets"]
    plotresonances = params['plotresonances'] # whether we should draw circles for resonances
    color = params['color']                       # colors for each of the particles (including the star)
    fscale= params['figscale']
    
    if sim.t < transits.events[0]['time']:        # only plot particles if the first transit has
        plotparticles = []
        show_orbit = False
    else:
        show_orbit = True
        
    # First make a color list that only includes the particles we want to plot
    #coloriterator = [color[i] for i in plotparticles]
    coloriterator = False
    lw=2 # linewidth
    
    
    fig = rebound.OrbitPlot(sim, figsize=(8,8), show_orbit=show_orbit, color='black', lw=lw, plotparticles=plotparticles)
    #fig = rebound.OrbitPlot(sim, figsize=(8,8), show_orbit=show_orbit, color='black', lw=lw, plotparticles=plotparticles,lim=fscale)
    #fig = rebound.OrbitPlot(sim, figsize=(8,8), color=coloriterator, lw=lw, plotparticles=plotparticles,lim=figwidth)

    ax = fig.axes[0] # turn off axes
    ax.axis('off')
    
    if show_orbit:
        refsize=25*lw # this is what REBOUND uses for size of circles in call to plt.scatter
        for i in plotparticles:
            if i==1:
                refsize=50
                c='black'
            if i==2:
                refsize=400
                c='black'
            if i>2:  #for white spaces
                refsize=3000
                c='white'
            ax.scatter(ps[i].x, ps[i].y, s=refsize, color=c, marker='o', zorder=4)
    if plottransits:
        refsize=25*lw # this is what REBOUND uses for size of circles in call to plt.scatter
                      # we want markers to match that when far from transit
        for i in transittargets: # overplot markers so planets have their own color
            #ax.scatter(ps[i].x, ps[i].y, s=refsize, color=color[i], marker='o', zorder=4)
            scale=ps[i].a/3     # length scale for making dots bigger
            if i==1:
                size=refsize
                c='black'
            if i==2:
                size=refsize
                c='crimson'
            if ps[i].x > 0 and np.abs(ps[i].y)/scale < 1: # increase size when particle is within `scale` of y=0, and on right
                size *= 1+15*np.exp(-np.abs(ps[i].y)/scale)
                ax.scatter(ps[i].x, ps[i].y, s=size, color=c, marker='o', zorder=5)
                
    if plotresonances:    
        fadetimescale = sim.particles[2].P/8. # timescale for the circles to dissapear
        # Find all the resonances that have already happened and are within fadetimescale of sim.t
        nearby_resonances = [resonance for resonance in resonances.events if sim.t - resonance['time'] < fadetimescale and sim.t - resonance['time'] > 0]
        nearby_transits = [transit for transit in transits.events if sim.t - transit['time'] < fadetimescale and sim.t - transit['time'] > 0]
        
        #add circle at 2:1 resonance for every time Janus is opposite to transit
        for resonance in nearby_resonances: 
            if resonance['target'] in resonances.targets: # or transit['target'] in resonances.targets 
            
                #print('Made it')
                # we now have to reload our binary and integrate to the time of resonance to get the x and y values of the inner planet to plot our line
                filename = "../binaries/janus.bin"
                sim2 = rebound.Simulation.from_file(filename)
                sim2.t = 0
                ss.rescale_time(sim2, sim2.particles[2].P)
                sim2.integrate(resonance['time'])
                r_res=sim2.particles[resonance['target']].a
                
                #Ellipse
                #e_res=sim2.particles[resonance['target']].e
                #pomega_res=sim2.particles[resonance['target']].pomega
                #print(r_res,e_res,pomega_res)
                #ell= Ellipse(xy=(0,0), width=2.*r_res, height=2.*r_res*np.sqrt(1.-e_res**2.),
                             #angle=pomega_res,color='r',linewidth=3,fill=False,alpha=max(1.-(sim.t-resonance['time'])/fadetimescale,0.))
                #ax.add_artist(ell)
                #plt.gcf().gca().add_artist(ell)
                
                circle2 = plt.Circle((0, 0), r_res*2.**(-2./3), color='crimson',linewidth=10,fill=False,alpha=max(1.-(sim.t-resonance['time'])/fadetimescale,0.))#fade not working
                ax.add_artist(circle2)
                plt.gcf().gca().add_artist(circle2)
                
        #add circle at 2:1 resonance for every Janus transit
        for transit in nearby_transits: 
            if transit['target'] in resonances.targets: 
            
                #print('Made it')
                # we now have to reload our binary and integrate to the time of resonance to get the x and y values of the inner planet to plot our line
                filename = "../binaries/janus.bin"
                sim2 = rebound.Simulation.from_file(filename)
                sim2.t = 0
                ss.rescale_time(sim2, sim2.particles[2].P)
                sim2.integrate(transit['time'])
                r_res=sim2.particles[transit['target']].a
                
                circle2 = plt.Circle((0, 0), r_res*2.**(-2./3), color='crimson',linewidth=10,fill=False,alpha=max(1.-(sim.t-transit['time'])/fadetimescale,0.))#fade not working
                ax.add_artist(circle2)
                plt.gcf().gca().add_artist(circle2)
                
    #bkg = imread('../images/US_background_image.png')
    #ax.imshow(bkg, zorder=0, extent=xlim+ylim)

    fig.savefig('tmp/pngs/{0:0=5d}.png'.format(params['frame_ctr']), transparent=False, dpi=300)
    plt.close(fig)  

Now we make our movie. We begin by importing what we need, and load our REBOUND binary simulation of the TRAPPIST-1 system that we want to animate/sonify. For convenience, we redefine the time in the simulation to 0, and rescale time so that every orbit of the outermost planet corresponds to one unit of time in the simulation (which we define as one beat in the music).

In [216]:
import sys
sys.path.append('../')
import systemsounds as ss
import numpy as np
import rebound

filename = "../binaries/janus.bin"
#filename = "../binaries/newsystem.bin"
sim = rebound.Simulation.from_file(filename)
sim.t = 0
print(sim.particles[2].P)
ss.rescale_time(sim, sim.particles[2].P) #rescale to Janus
print(sim.particles[2].P)
Periods=[sim.particles[i].P for i in range(1,3)]
e_s=[sim.particles[i].e for i in range(1,3)]
print('Epimethues, Janus Periods:',Periods)
print('Epimethues, Janus Eccentricity:',e_s)
#sim.dt = 1e-2
#sim.status()
'''
a_Janus=sim.particles[-1].a
N_ring=5
for i in range(N_ring):
    sim.add(m=0.,a=a_Janus*2**(-2./3),theta=2.*np.pi/N_ring*i)
'''

0.012074950561369303
0.9999999999999997
Epimethues, Janus Periods: [1.0029599257705506, 0.9999999999999997]
Epimethues, Janus Eccentricity: [0.004332915471032346, 0.005086661460336809]


'\na_Janus=sim.particles[-1].a\nN_ring=5\nfor i in range(N_ring):\n    sim.add(m=0.,a=a_Janus*2**(-2./3),theta=2.*np.pi/N_ring*i)\n'

We chose to begin the movie at 30 beats per minute. We have to convert this to simulation time per second. Above we rescaled time so that one orbit of the outer planet is one unit of simulation time, so 30 bpm / 60 = 0.5 beats per second. We also assign colors to each of the particles in the simulation.

In [217]:
bpm = 25  #aliasing issues with 20 and 32 particles

frames = ss.FrameRecorder(sim, time_per_sec=bpm/60)
transits = ss.EventRecorder(sim, lambda sim, i: sim.particles[i].y)
resonances = ss.EventRecorder(sim, lambda sim, i: -sim.particles[i].y) #2nd order resonance

frames.color = None #[None, 'black','black','black']
frames.figscale = 1.1*sim.particles[2].a

We also start our MIDI file now, so that we can enter the right tempo information as we go. We will add all the notes at the end:

In [218]:
from midiutil import MIDIFile

midifile = MIDIFile(adjust_origin=True)
midifile.addTempo(track=0, time=sim.t, tempo=bpm) 

# Test

In [219]:

frames.plottransits = True
frames.plotconjunctions = False
frames.plotresonances = True


planets = list(range(sim.N))

ps = planets[1:]
print("t = {0}\t planets = {1}".format(sim.t, ps))
frames.plotparticles = ps #don't plot Janus resonance as planet
transits.targets = planets[1:3]  
resonances.targets = [planets[2]]
frames.transittargets=transits.targets
#m=2. #resonant order (2:1)
#frames.r_ress=[sim.particles[1].a,sim.particles[-1].a]

bars=20
for i in range(bars):
    print("t = {0}\t planets = {1}".format(sim.t, ps))
    sim.integrate(sim.t+4)


t = 0.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 0.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 4.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 8.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 12.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 16.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 20.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 24.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 28.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 32.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 36.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 40.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 44.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 48.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 52.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 56.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 60.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 64.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 68.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 72.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 76.0	 planets = [1, 2, 3, 4, 5, 6, 7]


In [220]:
frames.plottransits = True
frames.plotresonances = False


planets = list(range(sim.N))
ps = planets[1:]
print("All moons: {0}".format(ps))

#Janus
frames.plotparticles = planets[2:3] 
transits.targets = [planets[2]]
resonances.targets = []
frames.transittargets=transits.targets
print("t = {0}\t moons = {1}".format(sim.t, frames.plotparticles))
sim.integrate(sim.t+2)

#Janus and Epi
frames.plotparticles = planets[1:3]
transits.targets = planets[1:3]  
resonances.targets = []
frames.transittargets=transits.targets
print("t = {0}\t moons = {1}".format(sim.t, frames.plotparticles))
sim.integrate(sim.t+2) #t0 1.8 then 4.2?


#All, with Janus triggers
frames.plotresonances = True
frames.plotparticles = ps 
resonances.targets = [planets[2]]
print("t = {0}\t moons = {1}".format(sim.t, frames.plotparticles))
sim.integrate(sim.t+4)


speedupbars = 3
barbpms = np.linspace(25,100,speedupbars+1)
bpms = []

N = 10 # number of gradations per bar for bpm adjustments (so it sounds smoother)   
for j, i in enumerate(range(speedupbars)):
    print("t = {0}\t bpm = {1}\t moons = {2}".format(sim.t, barbpms[j], ps))
    times = np.linspace(sim.t,sim.t+4,N,endpoint=True)
    bpms = np.linspace(barbpms[j],barbpms[j+1],N,endpoint=True) # array of bpm values at N points within the current bar
    for time, bpm in zip(times, bpms):
        frames.time_per_sec = bpm/60
        midifile.addTempo(track=0, time=sim.t, tempo=bpm) 
        sim.integrate(time)
bars=1
for i in range(bars):
    print("t = {0}\t moons = {1}".format(sim.t, ps))
    sim.integrate(sim.t+4)
    

All moons: [1, 2, 3, 4, 5, 6, 7]
t = 80.0	 moons = [2]
t = 82.0	 moons = [1, 2]
t = 84.0	 moons = [1, 2, 3, 4, 5, 6, 7]
t = 88.0	 bpm = 25.0	 moons = [1, 2, 3, 4, 5, 6, 7]
t = 92.0	 bpm = 50.0	 moons = [1, 2, 3, 4, 5, 6, 7]
t = 96.0	 bpm = 75.0	 moons = [1, 2, 3, 4, 5, 6, 7]
t = 100.0	 moons = [1, 2, 3, 4, 5, 6, 7]


# MIDI File

We now write the MIDI file. We scale all transit notes to the outermost planet, which we assign to a C4 note (MIDI note 48).

In [221]:
transit_notes = ss.calc_midi_notes(sim.particles, ref_note=48, ref_ID=-1)

for transit in transits.events:
    if transit['target']==1:
        midifile.addNote(track=0, channel=1, pitch=50, time=transit['time'], duration=1, volume=100)
    if transit['target']==2:
        midifile.addNote(track=0, channel=1, pitch=48, time=transit['time'], duration=1, volume=100)
for resonance in resonances.events:
    midifile.addNote(track=0, channel=1, pitch=60, time=resonance['time'], duration=1, volume=100)
        
with open("./janus.mid", "wb") as f:
    midifile.writeFile(f)

# Write images to temporary folder for movie

Now we write the images. This is parallelized, but making matplotlib plots is slow, so this can take a couple hours! (the movie is 2 mins long).

In [222]:
%%time
from subprocess import call
call("rm -f tmp/pngs/*", shell=True)
pool = rebound.InterruptiblePool()
res = pool.map(write_png, frames.events)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Applications/anaconda/envs/venv/lib/python3.6/site-packages/IPython/core/magics/execution.py", line 1200, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 3, in <module>
  File "/Applications/anaconda/envs/venv/lib/python3.6/site-packages/rebound/interruptible_pool.py", line 74, in __init__
    initargs, **kwargs)
  File "/Applications/anaconda/envs/venv/lib/python3.6/multiprocessing/pool.py", line 168, in __init__
    self._repopulate_pool()
  File "/Applications/anaconda/envs/venv/lib/python3.6/multiprocessing/pool.py", line 233, in _repopulate_pool
    w.start()
  File "/Applications/anaconda/envs/venv/lib/python3.6/multiprocessing/process.py", line 105, in start
  File "/Applications/anaconda/envs/venv/lib/python3.6/multiprocessing/context.py", line 277, in _Popen
  File "/Applications/anaconda/envs/venv/lib/python3.6/multiprocessing/popen_fork.py", line 20, in __init__
  File "/Applications/anaconda/envs/venv/lib/python

OSError: [Errno 24] Too many open files

# Outputting a movie file

You now have all the frames for the movie in `systemsounds/jupyter_examples/tmp/pngs`, as well as the MIDI file `systemsounds/jupyter_examples/trappist.mid`, and can stitch them together into a movie using your favorite software. For some ideas and options, see [MovieEditingSoftware.ipynb](MovieEditingSoftware.ipynb)


In [223]:
moviename = 'Janus.mp4'
fps = 30
try:
    call("rm -f {0}".format(moviename), shell=True)
except:
    pass
call("ffmpeg -r {0} -i tmp/pngs/%05d.png -c:v libx264 -pix_fmt yuv420p {1}".format(fps, moviename), shell=True)
#call("open test.mp4", shell=True)

OSError: [Errno 24] Too many open files